In [5]:
import torch, sys, random
from torch import nn
import pytorch_lightning as pl
from pathlib import Path
import numpy as np
from copy import deepcopy
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader
from typing import Optional
import src.datasets as datasets
import src.models as models

In [6]:
main_dm = datasets.MainDataModule(datasets.TrainDataSet("data/ru_small.bin", "data/sample_locs.npy", 8))
test_ds = datasets.TestDataSet()
test_dl = DataLoader(test_ds, batch_size=8)

In [7]:
baseline_model = models.BaselineModel()

In [8]:
trainer = pl.Trainer(
    limit_train_batches=10000, max_epochs=2, accelerator="gpu", devices=1
)
trainer.fit(model=baseline_model, datamodule=main_dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name      | Type       | Params
-----------------------------------------
0 | embedding | Embedding  | 16.4 M
1 | encoder   | Sequential | 2.1 M 
-----------------------------------------
34.9 M    Trainable params
0         Non-trainable params
34.9 M    Total params
139.471   Total estimated model params size (MB)


Epoch 0:   3%|▎         | 45/1452 [00:05<02:46,  8.43it/s, loss=10.2, v_num=3]

/home/is/armin-sa/miniconda3/envs/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
with torch.no_grad():
    tmp = baseline_model(torch.full((2,), 3))
tmp.argmax()


tensor(3)